In [74]:
import pandas as pd
import numpy as np 
import utilities
import datetime
import time
import xgboost
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
import preprocess

In [75]:
### Function to calculate errors 
def get_errors(y_true,y_pred,model_name="Model"):   

    err_mae=mae(y_true,y_pred)
    err_rmse=np.sqrt(mse(y_true,y_pred))
    err_r2=r2(y_true,y_pred)
        
    #print(model_name+" MAE:"+str(err_mae)+" RMSE:"+str(err_rmse)+" R2:"+str(err_r2))
  
    return print(model_name+" MAE:"+str(err_mae)+" RMSE:"+str(err_rmse)+" R2:"+str(err_r2))



In [76]:
import pandas as pd
import re
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

# Create list for all elements dataset includes
def create_element_list(formula_list):
    all_elements_list = []
    for formula in formula_list:
        element_list = re.findall('[A-Z][^A-Z]*', formula)
        for elem in element_list:
            element, number = split_number(elem)
            if element not in all_elements_list:
                all_elements_list.append(element)

    return all_elements_list

# parses elements and number of occurences from element-number couple
def split_number(text):

    element = text.rstrip('0123456789')
    number = text[len(element):]
    if number == "":
        number = 1
    return element, int(number)


#counts number of element occurences from the formula list     
def count_elements(formula_list,all_elements_list):

    element_counts=[0]*len(all_elements_list)
    for formula in formula_list:
        element_list=re.findall('[A-Z][^A-Z]*', formula)
        for elem in element_list:
            element, number = split_number(elem)
            element_index=all_elements_list.index(element)
            element_counts[element_index]=element_counts[element_index]+1
    
    return element_counts

#parses element symbols and number of occurences from the formula list and create one hot vector    
def create_oneHotVector(formula_list,all_elements_list):

    vectors = []   
    for formula in formula_list:
        element_vector=[0]*len(all_elements_list)
        element_list=re.findall('[A-Z][^A-Z]*', formula)
        for elem in element_list:
            element, number = split_number(elem)
            try:
                element_index=all_elements_list.index(element)
                element_vector[element_index]=number
            except:
                print(element+ " not found in:"+ formula)
        vectors.append(element_vector)    
    oneHotVector_df=pd.DataFrame(index = formula_list, data=vectors, columns=all_elements_list)
    
    return oneHotVector_df


#parses element symbols and number of occurences from the formula list and create one hot vector    
def filter_by_elements(formula,all_elements_list):

    current_element_list=re.findall('[A-Z][^A-Z]*', formula)
    for elem in current_element_list:
        element, number = split_number(elem)
        if(element not in all_elements_list):
            return False
    
    return True

def has_element(formula,element):

    element_list=re.findall('[A-Z][^A-Z]*', formula)
    for elem in element_list:
        current_element, number = split_number(elem)
        if(current_element == element):
            return True
    
    return False


def plot_corr(data,size=(40,18)):
 
    corr = data.corr()
    plt.figure(figsize=size)
    sns.heatmap(np.round(corr,2), annot=True,center=0,cmap="RdBu_r")


In [77]:

import pandas as pd
import numpy as np 
from rdkit import Chem
import utilities
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import mordred
from mordred import Calculator, descriptors


def preprocess_train1(train_set_df,test_set_df):

    #Get test data to remove from the train set
    test_inchiKey_list = test_set_df["InChIKey"].values    
    train_smiles_list=[]###
    train_formula_list=[]
    train_logS_list=[]
    #train_smiles_list=[]
    train_mordred_descriptors=[]

    
    # Train data filter
    for index, row in train_set_df.iterrows():
        smiles=row['SMILES']
        logS=row['Solubility']
        inchiKey=row['InChIKey']
        mol=Chem.MolFromSmiles(smiles)
        mol=Chem.AddHs(mol)
        formula=Chem.rdMolDescriptors.CalcMolFormula(mol)
        formula=formula.replace("+","")
        formula=formula.replace("-","")
        
        if(inchiKey not in test_inchiKey_list):
            if("." not in smiles):
                if(has_element(formula,"C")):
                    if(("+" not in smiles) and ("-" not in smiles)):
                        train_formula_list.append(formula)
                        train_smiles_list.append(smiles)###
                        train_logS_list.append(logS)
                        train_mordred_descriptors.append(get_mordred(mol))

            
    print("Training set size after preprocessing:",len(train_logS_list))
      
    
    #Get Mordred descriptor names
    column_names=get_mordred(Chem.MolFromSmiles("CC"),True)+["LogS"]
    
    unfiltered_train_np=np.column_stack([train_mordred_descriptors,train_logS_list])
    unfiltered_train_df=pd.DataFrame(index=train_formula_list, data=unfiltered_train_np,columns=column_names)
    # Drop columns with nan (only MIN and MAX values of estates has nan values)
    unfiltered_train_df = unfiltered_train_df[unfiltered_train_df.columns.drop(list(unfiltered_train_df.filter(regex='MIN')))]
    unfiltered_train_df = unfiltered_train_df[unfiltered_train_df.columns.drop(list(unfiltered_train_df.filter(regex='MAX')))]
    # Remove columns having lower occurrences then the threshold
    remove_threshold=20
    drop_cols = unfiltered_train_df.columns[(unfiltered_train_df == 0).sum() > (unfiltered_train_df.shape[0]-remove_threshold) ]
    unfiltered_train_df.drop(drop_cols, axis = 1, inplace = True) 
    
    # Convert values to numeric
    train_df = unfiltered_train_df.apply(pd.to_numeric)
    
    #element count vector
    all_elements_list = create_element_list(train_formula_list)    
    element_vector_df=create_oneHotVector(train_formula_list,all_elements_list)
    element_vector_df["LogS"]=train_logS_list
    #element_vector_df["SMILES"]=train_smiles_list
    # Combine all descriptors for all and best descriptors
    train_descriptors = pd.concat([train_df.drop(columns=['LogS']),element_vector_df], axis=1)
    
    return train_descriptors,all_elements_list,train_smiles_list

def preprocess_test1(test_set_df,element_list):
    
    test_smiles_list=[]
    test_formula_list=[]
    test_logS_list=[]
    test_mordred_descriptors=[]
    
    # Test data filter
    for index, row in test_set_df.iterrows():
        smiles=row['SMILES']
        logS=row['Solubility']
        mol=Chem.MolFromSmiles(smiles)
        mol=Chem.AddHs(mol)
        formula=Chem.rdMolDescriptors.CalcMolFormula(mol)
        formula=formula.replace("+","")
        formula=formula.replace("-","")
        
        if("." not in smiles):
            if(has_element(formula,"C")):
                if(("+" not in smiles) and ("-" not in smiles)):
                    test_smiles_list.append(smiles)
                    test_formula_list.append(formula)
                    test_logS_list.append(logS)
                    test_mordred_descriptors.append(get_mordred(mol))
    
    print("Test size after preprocessing:",len(test_logS_list))
    
    #Get Mordred descriptor names
    column_names=get_mordred(Chem.MolFromSmiles("CC"),True)+["LogS"]
    
    unfiltered_test_np=np.column_stack([test_mordred_descriptors,test_logS_list])
    unfiltered_test_df=pd.DataFrame(index=test_formula_list, data=unfiltered_test_np,columns=column_names)
    
    # Create element count vector 
    test_element_vector_df=create_oneHotVector(test_formula_list,element_list)
    test_element_vector_df["LogS"]=test_logS_list
    
    # Combine all descriptors
    test_descriptors = pd.concat([unfiltered_test_df.drop(columns=['LogS']),test_element_vector_df], axis=1)
    
    return test_descriptors, test_smiles_list


def select_features_lasso(train_data,test_data):
    """
    Selects descriptors from training data then applies on both training and test data 
    """
    
    lasso = Lasso(alpha=0.01,max_iter=10000,random_state=1).fit(train_data.drop(columns=['LogS']), train_data['LogS'])
    model = SelectFromModel(lasso, prefit=True)
    X_new_lasso = model.transform(train_data.drop(columns=['LogS']))
    # Get back the kept features as a DataFrame with dropped columns as all 0s
    selected_features = pd.DataFrame(model.inverse_transform(X_new_lasso), index=train_data.drop(columns=['LogS']).index, columns=train_data.drop(columns=['LogS']).columns)
    # Dropped columns have values of all 0s, keep other columns 
    selected_columns_lasso = selected_features.columns[selected_features.var() != 0]
    
    selected_data_train = train_data[selected_columns_lasso]             
    selected_data_test = test_data[selected_columns_lasso]
    selected_data_test = selected_data_test.apply(pd.to_numeric)
    
    print(selected_data_train.columns)
    print("Total selected descriptors by LASSO:",len(selected_data_train.columns))
      
    return selected_data_train, selected_data_test


#returns mordred descriptor vector
def get_mordred(mol, desc_names=False):   
    """
    Generates predefined descriptors for given mol object(Rdkit)
    If desc_names is True then returns only the list of the descriptor name
    """
    
    calc1 = mordred.Calculator()    

    calc1.register(mordred.AtomCount.AtomCount("X"))
    calc1.register(mordred.AtomCount.AtomCount("HeavyAtom"))
    calc1.register(mordred.Aromatic.AromaticAtomsCount)
    
    calc1.register(mordred.HydrogenBond.HBondAcceptor)
    calc1.register(mordred.HydrogenBond.HBondDonor)
    calc1.register(mordred.RotatableBond.RotatableBondsCount)  
    calc1.register(mordred.BondCount.BondCount("any", False))
    calc1.register(mordred.Aromatic.AromaticBondsCount)  
    calc1.register(mordred.BondCount.BondCount("heavy", False))       
    calc1.register(mordred.BondCount.BondCount("single", False))
    calc1.register(mordred.BondCount.BondCount("double", False))
    calc1.register(mordred.BondCount.BondCount("triple", False))
      
    calc1.register(mordred.McGowanVolume.McGowanVolume)
    calc1.register(mordred.TopoPSA.TopoPSA(True))
    calc1.register(mordred.TopoPSA.TopoPSA(False))
    calc1.register(mordred.MoeType.LabuteASA)
    calc1.register(mordred.Polarizability.APol)
    calc1.register(mordred.Polarizability.BPol)
    calc1.register(mordred.AcidBase.AcidicGroupCount)
    calc1.register(mordred.AcidBase.BasicGroupCount)
    calc1.register(mordred.EccentricConnectivityIndex.EccentricConnectivityIndex)        
    calc1.register(mordred.TopologicalCharge.TopologicalCharge("raw",1))
    calc1.register(mordred.TopologicalCharge.TopologicalCharge("mean",1))
    
    calc1.register(mordred.SLogP)
    calc1.register(mordred.BertzCT.BertzCT)
    calc1.register(mordred.BalabanJ.BalabanJ)
    calc1.register(mordred.WienerIndex.WienerIndex(True))
    calc1.register(mordred.ZagrebIndex.ZagrebIndex(1,1))
    calc1.register(mordred.ABCIndex)
    
    calc1.register(mordred.RingCount.RingCount(None, False, False, None, None))
    calc1.register(mordred.RingCount.RingCount(None, False, False, None, True))
    calc1.register(mordred.RingCount.RingCount(None, False, False, True, None))
    calc1.register(mordred.RingCount.RingCount(None, False, False, True, True))
    calc1.register(mordred.RingCount.RingCount(None, False, False, False, None))
    calc1.register(mordred.RingCount.RingCount(None, False, True, None, None))

    calc1.register(mordred.EState)

        
# if desc_names is "True" returns only name list
    if(desc_names):
        name_list=[]
        for desc in calc1.descriptors:
            name_list.append(str(desc))
        return name_list
#        return list(calc1._name_dict.keys())
    else: 
        result = calc1(mol)
        return result._values


In [78]:
#start = time.time()
import pandas as pd
train_set_df = pd.read_csv("data/dataset-not-FA.csv") ### Sorkun paper train data 
test_set_df = pd.read_csv("data/dataset-E.csv")   ### Sorkun paper test data 

In [79]:
print(len(train_set_df))
print(len(test_set_df))

6154
1291


In [12]:
## Change when dataset change ---
#test_set_df.rename(columns={'smiles_canon': 'SMILES'}, inplace=True)
#train_set_df.rename(columns={'smiles_canon': 'SMILES'}, inplace=True)

In [80]:
from rdkit.Chem import MolStandardize
import joblib
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors,rdMolDescriptors

In [81]:
all_descriptors_train_df_all,element_list,smiles_list=preprocess_train1(train_set_df,test_set_df)
all_descriptors_test_df_all,test_smiles_list=preprocess_test1(test_set_df,element_list)

[21:23:34] WARNING: not removing hydrogen atom without neighbors
[21:23:34] WARNING: not removing hydrogen atom without neighbors
[21:23:35] WARNING: not removing hydrogen atom without neighbors
[21:23:39] WARNING: not removing hydrogen atom without neighbors
[21:23:39] WARNING: not removing hydrogen atom without neighbors
[21:23:39] WARNING: not removing hydrogen atom without neighbors
[21:23:41] WARNING: not removing hydrogen atom without neighbors
[21:23:45] WARNING: not removing hydrogen atom without neighbors
[21:23:55] WARNING: not removing hydrogen atom without neighbors
[21:23:57] WARNING: not removing hydrogen atom without neighbors
[21:24:02] WARNING: not removing hydrogen atom without neighbors
[21:24:02] WARNING: not removing hydrogen atom without neighbors


Training set size after preprocessing: 4399
Test size after preprocessing: 1290


In [15]:
#column_names = all_descriptors_train_df_all.columns.tolist()

# Print the list of column names
#print(column_names)


['nX', 'nHeavyAtom', 'nAromAtom', 'nHBAcc', 'nHBDon', 'nRot', 'nBonds', 'nAromBond', 'nBondsO', 'nBondsS', 'nBondsD', 'nBondsT', 'VMcGowan', 'TopoPSA(NO)', 'TopoPSA', 'LabuteASA', 'apol', 'bpol', 'nAcid', 'nBase', 'ECIndex', 'GGI1', 'JGI1', 'SLogP', 'SMR', 'BertzCT', 'BalabanJ', 'WPol', 'Zagreb1', 'ABC', 'ABCGG', 'nRing', 'nHRing', 'naRing', 'naHRing', 'nARing', 'nFRing', 'NsCH3', 'NdCH2', 'NssCH2', 'NtCH', 'NdsCH', 'NaaCH', 'NsssCH', 'NddC', 'NtsC', 'NdssC', 'NaasC', 'NaaaC', 'NssssC', 'NsNH2', 'NssNH', 'NaaNH', 'NtN', 'NdsN', 'NaaN', 'NsssN', 'NaasN', 'NsOH', 'NdO', 'NssO', 'NaaO', 'NsF', 'NdsssP', 'NdS', 'NssS', 'NaaS', 'NdssS', 'NddssS', 'NsCl', 'NsBr', 'NsI', 'SsCH3', 'SdCH2', 'SssCH2', 'StCH', 'SdsCH', 'SaaCH', 'SsssCH', 'SddC', 'StsC', 'SdssC', 'SaasC', 'SaaaC', 'SssssC', 'SsNH2', 'SssNH', 'SaaNH', 'StN', 'SdsN', 'SaaN', 'SsssN', 'SaasN', 'SsOH', 'SdO', 'SssO', 'SaaO', 'SsF', 'SdsssP', 'SdS', 'SssS', 'SaaS', 'SdssS', 'SddssS', 'SsCl', 'SsBr', 'SsI', 'C', 'H', 'Br', 'N', 'O', 'I'

In [83]:
selected_columns_train = all_descriptors_train_df_all[['nX', 'nHeavyAtom', 'nAromAtom', 'nHBAcc', 'nHBDon', 'nRot', 'nBonds', 'nAromBond', 'nBondsO', 'nBondsS', 'nBondsD', 'nBondsT', 'VMcGowan', 'TopoPSA(NO)', 'TopoPSA', 'LabuteASA', 'apol', 'bpol', 'nAcid', 'nBase', 'ECIndex', 'GGI1', 'JGI1', 'SLogP', 'SMR', 'BertzCT', 'BalabanJ', 'WPol', 'Zagreb1', 'ABC', 'ABCGG', 'nRing', 'nHRing', 'naRing', 'naHRing', 'nARing', 'nFRing', 'NsCH3', 'NdCH2', 'NssCH2', 'NtCH', 'NdsCH', 'NaaCH', 'NsssCH', 'NddC', 'NtsC', 'NdssC', 'NaasC', 'NaaaC', 'NssssC', 'NsNH2', 'NssNH', 'NaaNH', 'NtN', 'NdsN', 'NaaN', 'NsssN', 'NaasN', 'NsOH', 'NdO', 'NssO', 'NaaO', 'NsF', 'NdsssP', 'NdS', 'NssS', 'NaaS', 'NdssS', 'NddssS', 'NsCl', 'NsBr', 'NsI', 'SsCH3', 'SdCH2', 'SssCH2', 'StCH', 'SdsCH', 'SaaCH', 'SsssCH', 'SddC', 'StsC', 'SdssC', 'SaasC', 'SaaaC', 'SssssC', 'SsNH2', 'SssNH', 'SaaNH', 'StN', 'SdsN', 'SaaN', 'SsssN', 'SaasN', 'SsOH', 'SdO', 'SssO', 'SaaO', 'SsF', 'SdsssP', 'SdS', 'SssS', 'SaaS', 'SdssS', 'SddssS', 'SsCl', 'SsBr', 'SsI', 'C', 'H', 'Br', 'N', 'O', 'I', 'Cl', 'S', 'F', 'P', 'As', 'Si', 'Se', 'Sn', 'Hg', 'Ge']]


In [84]:
selected_columns_test = all_descriptors_test_df_all[['nX', 'nHeavyAtom', 'nAromAtom', 'nHBAcc', 'nHBDon', 'nRot', 'nBonds', 'nAromBond', 'nBondsO', 'nBondsS', 'nBondsD', 'nBondsT', 'VMcGowan', 'TopoPSA(NO)', 'TopoPSA', 'LabuteASA', 'apol', 'bpol', 'nAcid', 'nBase', 'ECIndex', 'GGI1', 'JGI1', 'SLogP', 'SMR', 'BertzCT', 'BalabanJ', 'WPol', 'Zagreb1', 'ABC', 'ABCGG', 'nRing', 'nHRing', 'naRing', 'naHRing', 'nARing', 'nFRing', 'NsCH3', 'NdCH2', 'NssCH2', 'NtCH', 'NdsCH', 'NaaCH', 'NsssCH', 'NddC', 'NtsC', 'NdssC', 'NaasC', 'NaaaC', 'NssssC', 'NsNH2', 'NssNH', 'NaaNH', 'NtN', 'NdsN', 'NaaN', 'NsssN', 'NaasN', 'NsOH', 'NdO', 'NssO', 'NaaO', 'NsF', 'NdsssP', 'NdS', 'NssS', 'NaaS', 'NdssS', 'NddssS', 'NsCl', 'NsBr', 'NsI', 'SsCH3', 'SdCH2', 'SssCH2', 'StCH', 'SdsCH', 'SaaCH', 'SsssCH', 'SddC', 'StsC', 'SdssC', 'SaasC', 'SaaaC', 'SssssC', 'SsNH2', 'SssNH', 'SaaNH', 'StN', 'SdsN', 'SaaN', 'SsssN', 'SaasN', 'SsOH', 'SdO', 'SssO', 'SaaO', 'SsF', 'SdsssP', 'SdS', 'SssS', 'SaaS', 'SdssS', 'SddssS', 'SsCl', 'SsBr', 'SsI', 'C', 'H', 'Br', 'N', 'O', 'I', 'Cl', 'S', 'F', 'P', 'As', 'Si', 'Se', 'Sn', 'Hg', 'Ge']]


In [86]:
log_train=all_descriptors_train_df_all[['LogS']]
log_test=all_descriptors_test_df_all[['LogS']]

In [87]:
df_reset_train = selected_columns_train.reset_index(drop=True)
df_reset_test = selected_columns_test.reset_index(drop=True)
df_log_train = log_train.reset_index(drop=True)
df_log_test = log_test.reset_index(drop=True)

In [88]:
### After preprocess taking out LogS values ...
train_logS_list=all_descriptors_train_df_all["LogS"]
test_logS_list=all_descriptors_test_df_all["LogS"]

In [89]:
### Making the dataframe of train and test smiles list ...
df_smiles_train = pd.DataFrame(smiles_list, columns=['smiles'])
df_smiles_test = pd.DataFrame(test_smiles_list, columns=['smiles'])

In [90]:
### ### Making the dataframe of train and test LogS 
df_train1 = train_logS_list.to_frame()
df_test1 = test_logS_list.to_frame()

In [91]:
### Resetting the index of the dataframe 
df_smiles_train.reset_index(drop=True, inplace=True)
df_smiles_test.reset_index(drop=True, inplace=True)
df_train1.reset_index(drop=True, inplace=True)
df_test1.reset_index(drop=True, inplace=True)

In [92]:
### combining the LogS and smiles 
df_train2 = pd.concat([df_smiles_train, df_train1], axis=1)
df_test2 = pd.concat([df_smiles_test, df_test1], axis=1)


In [93]:
### Changing the column name 
df_train2 = df_train2.rename(columns={'LogS': 'Solubility'})
df_test2 = df_test2.rename(columns={'LogS': 'Solubility'})


In [43]:
df_train2.to_csv('data/train_sorkun.csv')
df_test2.to_csv('data/test_sorkun.csv')

In [94]:
### Selecting the important features with Lasso technique ...
selected_data_train, selected_data_test = select_features_lasso(all_descriptors_train_df_all,all_descriptors_test_df_all)


Index(['nHeavyAtom', 'nHBAcc', 'nHBDon', 'nRot', 'nBonds', 'nBondsO',
       'nBondsS', 'nBondsD', 'TopoPSA(NO)', 'TopoPSA', 'LabuteASA', 'bpol',
       'nAcid', 'nBase', 'ECIndex', 'GGI1', 'SLogP', 'SMR', 'BertzCT',
       'BalabanJ', 'WPol', 'Zagreb1', 'ABCGG', 'nHRing', 'naHRing', 'NsCH3',
       'NssCH2', 'NaaCH', 'NaaaC', 'NssssC', 'SsCH3', 'SdCH2', 'SssCH2',
       'StCH', 'SdsCH', 'SaaCH', 'SsssCH', 'StsC', 'SdssC', 'SaasC', 'SaaaC',
       'SssssC', 'SsNH2', 'SssNH', 'SaaN', 'SsssN', 'SaasN', 'SsOH', 'SdO',
       'SssO', 'SaaO', 'SsF', 'SdsssP', 'SdS', 'SddssS', 'SsCl', 'SsI', 'C'],
      dtype='object')
Total selected descriptors by LASSO: 58


/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 280.05010468922364, tolerance: 2.41512211911891
  model = cd_fast.enet_coordinate_descent(


In [95]:
from sklearn.model_selection import KFold
import time 
initial_start = time.time()
full_test=[]
full_pred_mlp=[]
full_pred_xgb=[]
full_pred_rf=[]
full_pred_consensus=[]
full_smiles_list=[]


#Run cross validation over the test set
#for LOO ---> n_splits=len(selected_data_test)
i=0
kf = KFold(n_splits=4,shuffle=True,random_state=1)
for train, test in kf.split(selected_data_test):   
    
    
    start = time.time()
    i=i+1
    print("\n Fold " + str(i))
    kfold_data_test=selected_data_test.iloc[test,:]
    kfold_data_train=selected_data_test.iloc[train,:]
 
    kfold_logS_test=test_logS_list[test]
    kfold_logS_train=test_logS_list[train]
    
    merged_data_train=pd.concat([selected_data_train,kfold_data_train], axis=0)
    merged_logS=pd.concat([train_logS_list,kfold_logS_train], axis=0)
    
    #Train and Test with Neural Nets
    mlp_model = MLPRegressor(activation='tanh', hidden_layer_sizes=(500), max_iter=500, random_state=0, solver='adam')
    mlp_model.fit(merged_data_train, merged_logS)   
    pred_mlp = mlp_model.predict(kfold_data_test)
    get_errors(kfold_logS_test,pred_mlp,"Neural Nets")
    
    #Train and Test with XGBoost
    xgboost_model = xgboost.XGBRegressor(n_estimators=1000) 
    xgboost_model.fit(merged_data_train, merged_logS)    
    pred_xgb = xgboost_model.predict(kfold_data_test)
    get_errors(kfold_logS_test,pred_xgb,"XGBoost")
    
    #Train and Test with Random Forest
    rf_model = RandomForestRegressor(random_state=0, n_estimators=1000) 
    rf_model.fit(merged_data_train, merged_logS)   
    pred_rf = rf_model.predict(kfold_data_test)   
    get_errors(kfold_logS_test,pred_rf,"Random Forest")
    
    #Calculate Consensus results
    pred_consensus=(pred_mlp+pred_xgb+pred_rf)/3
    get_errors(kfold_logS_test,pred_consensus,"Consensus")
   
    #Append results of the fold
    full_test = full_test + list(kfold_logS_test)
    full_pred_mlp = full_pred_mlp + list(pred_mlp)
    full_pred_xgb = full_pred_xgb + list(pred_xgb)
    full_pred_rf = full_pred_rf + list(pred_rf)
    full_pred_consensus = full_pred_consensus + list(pred_consensus)
    full_smiles_list = full_smiles_list + list(map(test_smiles_list.__getitem__, test))
    
    time_counsumed = time.time()-start
    print("Time used:", time_counsumed)  
    
 
print("\n Full Results")       
get_errors(full_test,full_pred_mlp,"Neural Nets")
get_errors(full_test,full_pred_xgb,"XGBoost")
get_errors(full_test,full_pred_rf,"Random Forest")
get_errors(full_test,full_pred_consensus,"Consensus")


time_counsumed = time.time()-initial_start 
print("Total time used:", time_counsumed)  



 Fold 1
Neural Nets MAE:0.487230862251057 RMSE:0.6421915868897696 R2:0.9120505754257605
XGBoost MAE:0.3985751755644016 RMSE:0.5617653137995053 R2:0.9327002239216711
Random Forest MAE:0.3845999730565278 RMSE:0.5193496342960305 R2:0.9424793958124674
Consensus MAE:0.3828557156111117 RMSE:0.5091221312673578 R2:0.9447225843094502
Time used: 97.59952473640442

 Fold 2
Neural Nets MAE:0.4210255240235784 RMSE:0.5709435534715755 R2:0.9131868972164727
XGBoost MAE:0.35946805663210785 RMSE:0.507997512953898 R2:0.9312738361215013
Random Forest MAE:0.34418057334747265 RMSE:0.48131717684112174 R2:0.9383033401694418
Consensus MAE:0.3295160022070942 RMSE:0.4546204891681147 R2:0.9449576533581984
Time used: 100.1931209564209

 Fold 3
Neural Nets MAE:0.4394967153391214 RMSE:0.5984188172635517 R2:0.9043913369703835
XGBoost MAE:0.36990282645906025 RMSE:0.555313254707663 R2:0.9176691040237809
Random Forest MAE:0.390359481348353 RMSE:0.5606467306376186 R2:0.9160800241609214
Consensus MAE:0.36460522646353316 

In [25]:
### Now we check.. is there any mathcing data with train data in test data ?

In [96]:
 ### Function to make smiles to canonical smiles ( Unique representation of the smiles )
 # Various Representation      Unique Representation 
   #OC(=O)C(Br)(Cl)N	         NC(Cl)(Br)C(=O)O
   #ClC(Br)(N)C(=O)O	         NC(Cl)(Br)C(=O)O
   #O=C(O)C(N)(Br)Cl	         NC(Cl)(Br)C(=O)O
#canonical_smiles_with_stereochemistry = Chem.MolToSmiles(mol, isomericSmiles=True)

from rdkit.Chem import MolFromSmiles as smi2mol
from rdkit.Chem import MolToSmiles as mol2smi
## Function to create canonical smiles 
def canon(smi):
    try:
        mol=smi2mol(smi, sanitize=True)
        smi_canon=mol2smi(mol, isomericSmiles=False, canonical=True)
        #smi_canon=mol2smi(mol, isomericSmiles=True,canonical=True) ### According to Sorkun 
        return(smi_canon)
    except:
        print("ERROR")
        return(smi)

In [97]:
### Applying function to create the column with canonical smiles.  
df_train2['smiles_canon'] = [canon(smi) for smi in df_train2.smiles]
df_test2['smiles_canon'] = [canon(smi) for smi in df_test2.smiles]

In [98]:
### Applying function to create the column with canonical smiles.  
#concatenated_df_train['smiles_canon'] = [canon(smi) for smi in concatenated_df_train.SMILES]
#concatenated_df_test['smiles_canon'] = [canon(smi) for smi in concatenated_df_test.SMILES]

In [99]:
### Finding the matching rows train and test data based on canonical smiles(Unique representation pf the smiles )
Match_rows = pd.merge(df_test2, df_train2, on=['smiles_canon'], how='inner')
print(len(Match_rows))
Match_rows
#Match_rows.to_csv('data/overlap_data.csv', index=False)



133


,smiles_x,Solubility_x,smiles_canon,smiles_y,Solubility_y
0,CCC=CC,-2.54,CC=CCC,CC\C=C/C,-2.5380
1,CCC=CC,-2.54,CC=CCC,CC\C=C\C,-2.5380
2,CCCCC=CC,-3.82,CC=CCCCC,CCCC\C=C\C,-3.8160
3,C(=CCC(C(=C)C)C1)(C1)C,-4.00,C=C(C)C1CC=C(C)CC1,CC(=C)[C@@H]1CCC(=CC1)C,-3.9940
4,ClC=CCl,-1.30,ClC=CCl,Cl\C=C\Cl,-1.1870
...,...,...,...,...,...
128,C1C(=O)C=C2CCC3C4CCC(O)(C#C)C4(C)CCC3C2(C)C1,-5.66,C#CC1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3CCC21C,C[C@]12CCC(=O)C=C1CC[C@@H]3[C@@H]2CC[C@@]4(C)[...,-5.6580
129,NCCC(O)C(=O)NC2CC(N)C(OC1OC(CN)C(O)C(O)C1O)C(O...,-0.50,NCCC(O)C(=O)NC1CC(N)C(OC2OC(CN)C(O)C(O)C2O)C(O...,NCC[C@H](O)C(=O)N[C@@H]1C[C@H](N)[C@@H](O[C@H]...,-0.5004
130,C1C(O)CC2=CCC3C4CC5OC6(OCC(C)CC6)C(C)C5C4(C)CC...,-7.32,CC1CCC2(OC1)OC1CC3C4CC=C5CC(O)CCC5(C)C4CCC3(C)...,C[C@@H]1CC[C@@]2(OC1)O[C@H]3C[C@H]4[C@@H]5CC=C...,-7.3170
131,C1C(=O)C=C2CCC3C4CCC(O)C4(C)CCC3C2(C)C1,-4.09,CC12CCC(=O)C=C1CCC1C2CCC2(C)C(O)CCC12,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@]34C)[C@...,-4.0910


In [100]:
condition = df_train2['smiles'] == 'CC\C=C/C'
filtered_df = df_train2[condition]

In [25]:
Match_rows.to_csv('data/overlap_data_new1.csv')

In [101]:
### to Cross check filter the data with matching smiles  
df_train_filter=df_train2[df_train2['smiles_canon']=='CC=CCC']

In [102]:
### to Cross check filter the data with matching smiles  
df_test_filter=df_test2[df_test2['smiles_canon']=='CC=CCC']

In [103]:
### cross checking for duplicates
df_test_filter

,smiles,Solubility,smiles_canon
12,CCC=CC,-2.54,CC=CCC


In [104]:
### cross checking for duplicates
df_train_filter

,smiles,Solubility,smiles_canon
2113,CC\C=C/C,-2.538,CC=CCC
2273,CC\C=C\C,-2.538,CC=CCC


In [105]:
### Finding the matching smiles in the train dataset..
cond = df_train2['smiles_canon'].isin(Match_rows['smiles_canon'])

#### Dropping the smiles which is same and making dataframe with uniques smiles 
# and making sure no same smiles in train  dataset and  test dataset 
df_train2.drop(df_train2[cond].index, inplace = True)

In [106]:
## Re setting the index
df_train2= df_train2.reset_index(drop=True)
### After removing the matching smiles from train dataset ... 
df_train2.shape

(4275, 3)

In [107]:
###Again cross check to find the mathcing smiles 
Match_rows = pd.merge(df_test2, df_train2, on=['smiles_canon'], how='inner')
print(len(Match_rows))
Match_rows

0


,smiles_x,Solubility_x,smiles_canon,smiles_y,Solubility_y


### Now we can comnfirm that there is no matching smiles in train dataset which match with test dataset

In [108]:
print(len(df_train2)) 
print(len(df_test2)) 

4275
1290


In [110]:
### Function to create inchikey from smiles
import pandas as pd
from rdkit import Chem
from rdkit.Chem import inchi

def smiles_to_inchikey(smiles):
    """
    Convert a SMILES string to an InChIKey.

    Parameters:
    - smiles (str): SMILES string.

    Returns:
    - str: InChIKey.
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
           inchi_str = inchi.MolToInchi(mol)
           inchikey = inchi.InchiToInchiKey(inchi_str)
           return inchikey
        else:
            return None
    except Exception as e:
        # Handle the exception (e.g., print a message) and return a default value
        print(f"Error processing SMILES: {smiles}. Error: {e}")
        return None

In [111]:
### Creating the Inchikey column with  
df_train2['InChIKey'] = df_train2['smiles_canon'].apply(lambda x: smiles_to_inchikey(x))
df_test2['InChIKey'] = df_test2['smiles_canon'].apply(lambda x: smiles_to_inchikey(x))

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Charges were rearranged

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Omitted undefined stereo

[22:00:18] WARNING: Charges were re

In [112]:
df_train2.rename(columns={'smiles': 'SMILES'}, inplace=True)
df_test2.rename(columns={'smiles': 'SMILES'}, inplace=True)

df_train2.rename(columns={'LogS': 'Solubility'}, inplace=True)
df_test2.rename(columns={'LogS': 'Solubility'}, inplace=True)

In [113]:
all_descriptors_train_df_all,element_list,smiles_list=preprocess_train1(df_train2,df_test2)
all_descriptors_test_df_all,test_smiles_list=preprocess_test1(df_test2,element_list)

Training set size after preprocessing: 4274
Test size after preprocessing: 1290


In [114]:
selected_data_train1, selected_data_test1 = select_features_lasso(all_descriptors_train_df_all,all_descriptors_test_df_all)


Index(['nHeavyAtom', 'nHBAcc', 'nHBDon', 'nRot', 'nBonds', 'nBondsO',
       'nBondsS', 'nBondsD', 'TopoPSA(NO)', 'TopoPSA', 'LabuteASA', 'bpol',
       'nAcid', 'nBase', 'ECIndex', 'GGI1', 'SLogP', 'SMR', 'BertzCT',
       'BalabanJ', 'WPol', 'Zagreb1', 'ABCGG', 'naHRing', 'NsCH3', 'NssCH2',
       'NaaCH', 'NaaaC', 'NssssC', 'SsCH3', 'SdCH2', 'SssCH2', 'StCH', 'SdsCH',
       'SaaCH', 'SsssCH', 'StsC', 'SaasC', 'SaaaC', 'SssssC', 'SsNH2', 'SssNH',
       'SaaN', 'SsssN', 'SaasN', 'SsOH', 'SdO', 'SssO', 'SaaO', 'SsF',
       'SdsssP', 'SdS', 'SddssS', 'SsCl', 'SsI', 'C'],
      dtype='object')
Total selected descriptors by LASSO: 56


/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 318.14272803833774, tolerance: 2.3817965403675414
  model = cd_fast.enet_coordinate_descent(


In [115]:
train_logS_list=all_descriptors_train_df_all["LogS"]
test_logS_list=all_descriptors_test_df_all["LogS"]

In [116]:
### Now running the same way with removing matching smiles 

from sklearn.model_selection import KFold
import time 
initial_start = time.time()
full_test=[]
full_pred_mlp=[]
full_pred_xgb=[]
full_pred_rf=[]
full_pred_consensus=[]
full_smiles_list=[]


#Run cross validation over the test set
#for LOO ---> n_splits=len(selected_data_test)
i=0
kf = KFold(n_splits=4,shuffle=True,random_state=1)
for train, test in kf.split(selected_data_test1):   
    
    
    start = time.time()
    i=i+1
    print("\n Fold " + str(i))
    kfold_data_test=selected_data_test1.iloc[test,:]
    kfold_data_train=selected_data_train1.iloc[train,:]
 
    kfold_logS_test=test_logS_list[test]
    kfold_logS_train=test_logS_list[train]
    
    merged_data_train=pd.concat([selected_data_train1,kfold_data_train], axis=0)
    merged_logS=pd.concat([train_logS_list,kfold_logS_train], axis=0)
    
    #Train and Test with Neural Nets
    mlp_model = MLPRegressor(activation='tanh', hidden_layer_sizes=(500), max_iter=500, random_state=0, solver='adam')
    mlp_model.fit(merged_data_train, merged_logS)   
    pred_mlp = mlp_model.predict(kfold_data_test)
    get_errors(kfold_logS_test,pred_mlp,"Neural Nets")
    
    #Train and Test with XGBoost
    xgboost_model = xgboost.XGBRegressor(n_estimators=1000) 
    xgboost_model.fit(merged_data_train, merged_logS)    
    pred_xgb = xgboost_model.predict(kfold_data_test)
    get_errors(kfold_logS_test,pred_xgb,"XGBoost")
    
    #Train and Test with Random Forest
    rf_model = RandomForestRegressor(random_state=0, n_estimators=1000) 
    rf_model.fit(merged_data_train, merged_logS)   
    pred_rf = rf_model.predict(kfold_data_test)   
    get_errors(kfold_logS_test,pred_rf,"Random Forest")
    
    #Calculate Consensus results
    pred_consensus=(pred_mlp+pred_xgb+pred_rf)/3
    get_errors(kfold_logS_test,pred_consensus,"Consensus")
   
    #Append results of the fold
    full_test = full_test + list(kfold_logS_test)
    full_pred_mlp = full_pred_mlp + list(pred_mlp)
    full_pred_xgb = full_pred_xgb + list(pred_xgb)
    full_pred_rf = full_pred_rf + list(pred_rf)
    full_pred_consensus = full_pred_consensus + list(pred_consensus)
    full_smiles_list = full_smiles_list + list(map(test_smiles_list.__getitem__, test))
    
    time_counsumed = time.time()-start
    print("Time used:", time_counsumed)  
    
 
print("\n Full Results")       
get_errors(full_test,full_pred_mlp,"Neural Nets")
get_errors(full_test,full_pred_xgb,"XGBoost")
get_errors(full_test,full_pred_rf,"Random Forest")
get_errors(full_test,full_pred_consensus,"Consensus")


time_counsumed = time.time()-initial_start 
print("Total time used:", time_counsumed)  



 Fold 1
Neural Nets MAE:0.6087158391412425 RMSE:0.7790291353950112 R2:0.8705770842594465
XGBoost MAE:0.6299207725002208 RMSE:0.87938466800007 R2:0.8350844688752803
Random Forest MAE:0.5991422647948416 RMSE:0.8369289585172786 R2:0.8506239506784348
Consensus MAE:0.555098115063902 RMSE:0.740886929663045 R2:0.8829402356851863
Time used: 90.85748505592346

 Fold 2
Neural Nets MAE:0.6034553110125563 RMSE:0.7810219921305688 R2:0.8375478523574229
XGBoost MAE:0.5919366652896807 RMSE:0.8261187388752006 R2:0.8182460416435219
Random Forest MAE:0.49422066804346065 RMSE:0.7071319741226297 R2:0.8668319987868249
Consensus MAE:0.49647695356962307 RMSE:0.6846993217501071 R2:0.8751470734863156
Time used: 91.44238471984863

 Fold 3
Neural Nets MAE:0.6596391048500405 RMSE:0.8541611451647058 R2:0.8052101217255796
XGBoost MAE:0.5901921565749 RMSE:0.8161768104008609 R2:0.8221494286422081
Random Forest MAE:0.5586584726587384 RMSE:0.7799581088869051 R2:0.837583810380621
Consensus MAE:0.541916249841211 RMSE:0.7